import statements, etc

In [ ]:
!mkdir -p processedmidi

In [ ]:
!pip install mido

In [ ]:
!pip install midiutil

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import numpy as np
import mido
from midiutil import MIDIFile
import io

In [ ]:
# input_melody = "59 - 59 - 62 - 62 - 59 - 59 - 62 - 62 - 66 - 66 - 62 - 62 - 66 - 66 - 69 - 69 -"

In [ ]:
test_melodies = [
    "_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 59 - 62 - 62 - 74 - 59 - 62 - 62 - 66 -",
    "59 - 62 - 62 - 74 - 59 - 62 - 62 - 66 - 62 - 78 - 62 - 66 - 66 - 69 - 62 - 81 -",
    "62 - 78 - 62 - 66 - 66 - 69 - 62 - 81 - 66 - 69 - 50 - 62 - - - 66 - - - 62 -",
    "66 - 69 - 50 - 62 - - - 66 - - - 62 - 50 - 62 - - - 66 - - - 62 - 50 - 62 -",
    "50 - 62 - - - 66 - - - 62 - 50 - 62 - - - 66 - - - 57 - 50 - 62 - - - 66 -",
    "- - 66 - - - 57 - 50 - 62 - - - 66 - - - 62 - 59 - 62 - 62 - 74 - 59 - 62 -",
    "- - 62 - 59 - 62 - 62 - 74 - 59 - 62 - 62 - 66 - 62 - 78 - 62 - 66 - 66 - 69 -",
    "62 - 66 - 62 - 78 - 62 - 66 - 66 - 69 - 62 - 81 - 66 - 69 - 50 - 62 - - - 65 -",
    "62 - 81 - 66 - 69 - 50 - 62 - - - 65 - - - 62 - 50 - 62 - - - 65 - - - 62 -",
    "- - 62 - 50 - 62 - - - 65 - - - 62 - 50 - 62 - - - 65 - - - 62 - 50 - 62 -",
    "50 - 62 - - - 65 - - - 62 - 50 - 62 - - - 65 - - - 62 - 59 - 62 - 62 - 74 -",
    "- - 65 - - - 62 - 59 - 62 - 62 - 74 - 59 - 62 - 62 - 66 - 62 - 78 - 62 - 66 -",
    "59 - 62 - 62 - 66 - 62 - 78 - 62 - 66 - 66 - 69 - 62 - 81 - 66 - 69 - 62 - 66 -",
    "66 - 69 - 62 - 81 - 66 - 69 - 62 - 66 - 62 - 73 - 62 - 66 - 62 - 66 - 62 - 73 -",
    "62 - 73 - 62 - 66 - 62 - 66 - 62 - 73 - 62 - 66 - 62 - 66 - 62 - 73 - 62 - 66 -",
    "62 - 66 - 62 - 66 - 62 - 73 - 62 - 66 - 62 - 66 - 62 - 73 - 62 - 66 - 59 - 62 -",
    "62 - 66 - 62 - 73 - 62 - 66 - 59 - 62 - 62 - 74 - 59 - 62 - 62 - 66 - 62 - 78 -",
    "62 - 74 - 59 - 62 - 62 - 66 - 62 - 78 - 62 - 66 - 66 - 69 - 62 - 81 - 66 - 69 -",
    "62 - 66 - 66 - 69 - 62 - 81 - 66 - 69 - 62 - 65 - 62 - 71 - 62 - 65 - 62 - 65 -",
    "62 - 65 - 62 - 71 - 62 - 65 - 62 - 65 - 62 - 71 - 62 - 65 - 62 - 65 - 62 - 71 -",
    "62 - 71 - 62 - 65 - 62 - 65 - 62 - 71 - 62 - 65 - 62 - 65 - 62 - 71 - 62 - 65 -",
    "62 - 65 - 62 - 65 - 62 - 71 - 62 - 65 - 59 - 62 - 62 - 74 - 59 - 62 - 62 - 66 -",
    "59 - 62 - 62 - 74 - 59 - 62 - 62 - 66 - 62 - 78 - 62 - 66 - 66 - 69 - 62 - 81 -",
    "62 - 78 - 62 - 66 - 66 - 69 - 62 - 81 - 66 - 69 - 50 - 62 - - - 66 - - - 62 -",
    "66 - 69 - 50 - 62 - - - 66 - - - 62 - 50 - 62 - - - 66 - - - 62 - 50 - 62 -",
    "50 - 62 - - - 66 - - - 62 - 50 - 62 - - - 66 - - - 57 - 50 - 62 - - - 66 -",
    "- - 66 - - - 57 - 50 - 62 - - - 66 - - - 62 - 59 - 62 - 62 - 74 - 59 - 62 -",
    "- - 62 - 59 - 62 - 62 - 74 - 59 - 62 - 62 - 66 - 62 - 78 - 62 - 66 - 66 - 69 -",
    "62 - 66 - 62 - 78 - 62 - 66 - 66 - 69 - 62 - 81 - 66 - 69 - 50 - 62 - - - 65 -",
    "62 - 81 - 66 - 69 - 50 - 62 - - - 65 - - - 62 - 50 - 62 - - - 65 - - - 62 -",
    "- - 62 - 50 - 62 - - - 65 - - - 62 - 50 - 62 - - - 65 - - - 62 - 50 - 62 -",
    "50 - 62 - - - 65 - - - 62 - 50 - 62 - - - 65 - - - 62 - 59 - 62 - 62 - 74 -",
    "- - 65 - - - 62 - 59 - 62 - 62 - 74 - 59 - 62 - 62 - 66 - 62 - 78 - 62 - 66 -",
    "59 - 62 - 62 - 66 - 62 - 78 - 62 - 66 - 66 - 69 - 62 - 81 - 66 - 69 - 62 - 66 -",
    "66 - 69 - 62 - 81 - 66 - 69 - 62 - 66 - 62 - 73 - 62 - 66 - 62 - 66 - 62 - 73 -",
    "62 - 73 - 62 - 66 - 62 - 66 - 62 - 73 - 62 - 66 - 62 - 66 - 62 - 73 - 62 - 66 -",
    "62 - 66 - 62 - 66 - 62 - 73 - 62 - 66 - 62 - 66 - 62 - 73 - 62 - 66 - 59 - 62 -",
    "62 - 66 - 62 - 73 - 62 - 66 - 59 - 62 - 62 - 74 - 59 - 62 - 62 - 66 - 62 - 78 -",
    "62 - 74 - 59 - 62 - 62 - 66 - 62 - 78 - 62 - 66 - 66 - 69 - 62 - 81 - 66 - 69 -",
    "62 - 66 - 66 - 69 - 62 - 81 - 66 - 69 - 62 - 65 - 62 - 71 - 62 - 65 - 62 - 65 -",
    "62 - 65 - 62 - 71 - 62 - 65 - 62 - 65 - 62 - 71 - 62 - 65 - 62 - 65 - 62 - 71 -",
    "62 - 71 - 62 - 65 - 62 - 65 - 62 - 71 - 62 - 65 - 62 - 65 - 62 - 71 - 62 - 65 -",
    "62 - 65 - 62 - 65 - 62 - 71 - 62 - 65 - 62 - 65 - 62 - 71 - 62 - 65 - 62 - 65 -",
    "62 - 65 - 62 - 71 - 62 - 65 - 62 - 65 - 62 - 71 - 62 - 65 - 62 - 65 - 62 - 71 -",
    "62 - 71 - 62 - 65 - 62 - 65 - 62 - 71 - 62 - 65 - 62 - 65 - 62 - 71 - 62 - 65 -",
    "62 - 65 - 62 - 65 - 62 - 71 - 62 - 65 - _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _"
]
#  46 in total

In [ ]:
# Load the model
model = load_model('20240923_generative_delay.h5')

# Load the tokenizer
with open('20240923_tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

function definitions - 16th note resolution

In [ ]:
def generate_melody(model, tokenizer, input_sequence, temperature=1.0):
    input_seq = tokenizer.texts_to_sequences([input_sequence])[0]
    input_seq = tf.keras.preprocessing.sequence.pad_sequences([input_seq], maxlen=32, truncating='pre')
    prediction = model.predict(input_seq, verbose=0)[0]
    # Apply temperature
    prediction = np.log(prediction) / temperature
    exp_preds = np.exp(prediction)
    prediction = exp_preds / np.sum(exp_preds, axis=-1, keepdims=True)
    output_seq = []
    for step in range(32):
        probas = prediction[step]
        choice = np.random.choice(len(probas), p=probas)
        output_seq.append(choice)
    output_melody = ' '.join([tokenizer.index_word.get(token, '') for token in output_seq])

#    print("Generated melody before formatting:")
#    print(output_melody)

    # Rotate melody so that the first symbol is a number
    melody_parts = output_melody.split()
    rotation_index = next((i for i, part in enumerate(melody_parts) if part.isdigit()), 0)
    rotated_melody = melody_parts[rotation_index:] + melody_parts[:rotation_index]

    # Replace "-" following "_" with the last numerical symbol
    last_num = None
    for i in range(len(rotated_melody)):
        if rotated_melody[i].isdigit():
            last_num = rotated_melody[i]
        elif i > 0 and rotated_melody[i-1] == '_' and rotated_melody[i] == '-' and last_num:
            rotated_melody[i] = last_num

    formatted_melody = ' '.join(rotated_melody)

    print("Generated melody with formatting applied:")
    print(formatted_melody)

    return formatted_melody

def text_to_midi(melody_text):
    # Initialize the MIDI file
    midi = MIDIFile(1)  # One track
    track = 0
    time = 0
    tempo = 110  # 110 BPM
    midi.addTempo(track, time, tempo)

    # Calculate the duration of a 16th note in beats
    beat_duration = 1 / 4  # A quarter note is 1 beat, so a 16th note is 1/4 beat

    # Parse the melody text
    notes = melody_text.split()
    current_note = None
    duration = 0

    for symbol in notes:
        if symbol.isdigit():
            # If there was a previous note, add it to the MIDI file
            if current_note is not None:
                midi.addNote(track, 0, current_note, time - duration, duration, 100)

            # Start a new note
            current_note = int(symbol)
            duration = beat_duration
            time += beat_duration
        elif symbol == "-":
            # Continue the current note
            if current_note is not None:
                duration += beat_duration
            time += beat_duration
        elif symbol == "_":
            # Silent note
            if current_note is not None:
                midi.addNote(track, 0, current_note, time - duration, duration, 100)
                current_note = None
            time += beat_duration
            duration = 0

    # Add the last note if there is one
    if current_note is not None:
        midi.addNote(track, 0, current_note, time - duration, duration, 100)

    # Create an in-memory file-like object
    midi_file = io.BytesIO()
    midi.writeFile(midi_file)
    midi_file.seek(0)

    return midi_file

generate melody with model

In [ ]:
# Generate a melody
# generated_melody = generate_melody(model, tokenizer, input_melody, temperature=0.7)

convert melody to midi

In [ ]:
# midi_file = text_to_midi(test_melody_2)

# save the MIDI file
# with open("processedmidi/test_melody_2.mid", "wb") as f:
#     f.write(midi_file.getvalue())

Automatically generate 10 new melodies for each of a bank of input melodies

In [ ]:
# Temperatures for generation
temperatures = [0.7, 0.8, 0.9]

# Generate and save test melodies
for i, melody in enumerate(test_melodies, 1):
    midi_file = text_to_midi(melody)
    with open(f"processedmidi/test_melody_{i:02d}.mid", "wb") as f:
        f.write(midi_file.getvalue())

# Generate and save generated melodies
for i, test_melody in enumerate(test_melodies, 1):
    for temp in temperatures:
        for j in range(10):
            generated_melody = generate_melody(model, tokenizer, test_melody, temperature=temp)
            midi_file = text_to_midi(generated_melody)

            # Calculate the correct number for the generated melody
            gen_num = ((i - 1) * 30) + (temperatures.index(temp) * 10) + j + 1

            with open(f"processedmidi/generated_melody_{gen_num:04d}.mid", "wb") as f:
                f.write(midi_file.getvalue())

print("All melodies generated and saved successfully!")

In [ ]:
import shutil

def zip_folder(folder_name):
    # The output zip file will have the same name as the folder
    shutil.make_archive(folder_name, 'zip', folder_name)

# Usage
zip_folder("processedmidi")